# Tutorial

Using simplified examples, we demonstrate how to use our functionality, when to use it, and the consequences of not reflecting the sampling design when making statistical inferences. We also provide side-by-side comparison to Stata results for robustness.

## When to use our functionality

The functionality is for unconstrained maximum-likelihood estimation of survey data. This means logit and probit estimations. Survey data take on many components; further elaborated here (insert link).

## How to use it

### Necessary objects 

To use our functionality, we need to define a few objects first. 

1) First, define your dataset as a pandas dataframe object. 

2) Create a design dictionary, where the keys acceptable keys are "psu", "strata", "weight", and "fpc". The values are the column names from the dataset that correspond to that key. For example, if you wish to cluster by school, you would define the key-value pair as {"psu": "school"}. Given the variety of survey data and what they provide for statistical inference, the design dictionary can be empty, contain only weight, only the primary sampling unit (also referred to as cluster) and/or other variations of the four options. 

3) Use the data and the design dictionary as arguments to create the design options dataframe. 

3) Define a formula, which is your model in string form. 

Below, we present an example of the necessary objects and four illustrations of the function in action. 

In [1]:
import pandas as pd
import os

In [2]:
from estimagic.inference.src.functions.se_estimation import likelihood_inference
data = pd.read_csv("data.csv")
design_dict = {"psu": "psu", "strata": "stratum", "weight": "weight"}
formula = ["eco_friendly ~ ppltrst + male + income"]
data.head(5)

,Unnamed: 0,ppltrst,agea,dweight,pspwght,pweight,psu,stratum,weight,male,eco_friendly,income
0,2,5.0,68.0,0.389058,0.315753,0.370393,164.0,88.0,0.116953,0.0,1.0,2.0
1,3,6.0,54.0,0.642594,0.472467,0.370393,562.0,24.0,0.174999,1.0,1.0,4.0
2,5,3.0,65.0,1.180579,1.011379,0.370393,459.0,71.0,0.374608,0.0,1.0,10.0
3,9,5.0,41.0,1.784959,0.997574,0.370393,113.0,25.0,0.369494,0.0,0.0,4.0
4,10,2.0,57.0,0.412953,0.550060,0.370393,311.0,63.0,0.203738,0.0,1.0,1.0


In [ ]:
def likelihood_inference(log_like_kwargs, design_dict=None, cov_type="opg"):
    """Pseudolikelihood estimation and inference.

    Args:
        log_like_kwargs (dict): Additional keyword arguments for the
            likelihood function.
            Example:
                log_like_kwargs = {
                    "formulas": equation,
                    "data": orig_data,
                    "model": "probit"
                }
        design_dict (dict): dicitonary containing specified design options
        cov_type (str): One of ["opg", "oim", "sandwich"]. opg and oim only
            work when *design_dict* is None. opg is default.

    Returns:
        params (pd.DataFrame): params that maximize likelihood
            - "standard_error"
            - "ci_lower"
            - "ci_upper"
        cov (pd.DataFrame): Covariance matrix of estimated parameters. Index and columns
            are the same as params.index.

    """
    design_options = design_options_preprocessing(log_like_kwargs["data"], design_dict)
    info, params = estimate_parameters(
        estimate_likelihood, design_options, log_like_kwargs, dashboard=False
    )
    like_se, like_var = choose_cov(
        params,
        log_like_kwargs["formulas"],
        log_like_kwargs["data"],
        log_like_kwargs["model"],
        design_dict,
        design_options,
        cov_type,
    )
    params_df, cov = inference_table(params, like_se, like_var, cov_type=cov_type)
    return params_df, cov

## Example 1
### Logit illustration with design options not specified

When design options are not specified, your model has access to three variance estimators: (1) Robust or "Sandwich" estimator (2) Observed Information Matrix (3) Outer Product of Gradients. These are explained in the background section. For brevity, we just change the design dictionary; keeping formula and data from above. By selecting no design options, you are declaring your data was generated through random sampling - thus, each observation is independent of one another. 

In [19]:
# Using formula and data from before. Running a logit model.
log_like_kwargs = {
                    "formulas": formula, 
                    "data": data, 
                    "model": "logit"
                }
params_df, cov = likelihood_inference(log_like_kwargs, design_dict=None, cov_type="sandwich")
# For logit with design options not specified, robust
correct_params_df = pd.DataFrame()
correct_params_df["value"] = [0.0109796, -0.0064468, -.1890401, .9659383]
correct_params_df["sandwich_standard_error"] = [.0067696, .0059065, .0315615, .0474801]
correct_params_df["ci_lower"] = [-.0022886, -.0180233, -.2508995, .8728789]
correct_params_df["ci_upper"] = [.0242478, .0051297, -.1271807, 1.058998]
correct_params_df, params_df

(      value  sandwich_standard_error  ci_lower  ci_upper
 0  0.010980                 0.006770 -0.002289  0.024248
 1 -0.006447                 0.005907 -0.018023  0.005130
 2 -0.189040                 0.031561 -0.250899 -0.127181
 3  0.965938                 0.047480  0.872879  1.058998,
               value  sandwich_standard_errors  ci_lower  ci_upper
 Intercept  0.965934                  0.047480  0.872873  1.058995
 ppltrst    0.010981                  0.006770 -0.002288  0.024249
 male      -0.189041                  0.031561 -0.250901 -0.127180
 income    -0.006447                  0.005906 -0.018024  0.005130)

## Example 2
### Logit illustration with only weight specified

Suppose you are attempting to establish some generalized effect across countries. You collect 2000 observations for each country. The survey method could be random sampling within countries, but you wish to weigh the data with a population weight. In the dataset, observations from smaller populations will be overrepresented compared to observations from larger countries, thus the population weight corrects by adding more weight to the observations from larger countries.

In [4]:
# Define design options in a dictionary. Our weight column is called "weight".
design_dict = {"weight": "weight"}
# Using formula and data from before. Running a weighted-logit model.
log_like_kwargs = {
                    "formulas": formula, 
                    "data": data, 
                    "model": "logit"
                }
params_df, cov = likelihood_inference(log_like_kwargs, design_dict=design_dict, cov_type="sandwich")

# For logit with only weight, robust estimator
correct_params_df = pd.DataFrame()
correct_params_df["value"] = [.0668702, -.0055281, -.1629321, .5978668]
correct_params_df["sandwich_standard_error"] = [.0150341, .0119522, .0634397, .094486]
correct_params_df["ci_lower"] = [.0374039, -.028954, -.2872715, .4126777]
correct_params_df["ci_upper"] = [.0963364, .0178977, -.0385926, .7830559]
correct_params_df, params_df

(      value  sandwich_standard_error  ci_lower  ci_upper
 0  0.066870                 0.015034  0.037404  0.096336
 1 -0.005528                 0.011952 -0.028954  0.017898
 2 -0.162932                 0.063440 -0.287272 -0.038593
 3  0.597867                 0.094486  0.412678  0.783056,
               value  sandwich_standard_errors  ci_lower  ci_upper
 Intercept  0.597865                  0.094486  0.412673  0.783057
 ppltrst    0.066872                  0.015034  0.037405  0.096339
 male      -0.162942                  0.063440 -0.287284 -0.038600
 income    -0.005529                  0.011952 -0.028955  0.017898)

Regardless of variance estimator, the parameters do not change. Yet, when a weight is defined, the pseudo-likelihood function is weighted, and thus, requires a new parameter vector which maximizes the function. This is why the parameters have changed compared to Example 1. The standard errors have also changed, more than doubling those of Example 1. 

## Example 3
### Probit illustration with psu, strata and weight specified

For the following illustration, we specify the primary sampling unit (psu), the strata and a weight. In case you have multiple weights, for example, a design weight and a population weight, you should generate another variable which is their product. Details on weights are here. In addition, in case strata have just one cluster, we use the "grand mean" method. More on this in the background. Finally, when clusters or strata are defined, only the robust or "sandwich" estimation is possible. 

[Population weights were applied to correct for population size the design weights are there to correct for unequal probabilities for selection due to the sampling design used — for example, using convenient sampling because it is cost-effective Post-stratification weights were applied to reduce sampling error further and correct for non-response bias; made possible by using auxiliary information. Auxiliary information includes age, gender, education, and region of the respondent]

In [6]:
design_dict = {"psu": "psu", "strata": "stratum", "weight": "weight"}
# Using formula and data from before. Running a weighted-probit model.
log_like_kwargs = {
                    "formulas": formula, 
                    "data": data, 
                    "model": "probit"
                }
params_df, cov = likelihood_inference(log_like_kwargs, design_dict=design_dict, cov_type="sandwich")

# For probit with psu, strata, and weight, robust
correct_params_df = pd.DataFrame()
correct_params_df["value"] = [.0400164, -.0030763, -.0978971, .3730149]
correct_params_df["sandwich_standard_error"] = [.009638, .0081467, .0375429, .0608428]
correct_params_df["ci_lower"] = [.021124, -.0190455, -.1714887, .2537509]
correct_params_df["ci_upper"] = [.0589088, .0128929, -.0243055, .4922788]
correct_params_df, params_df

(      value  sandwich_standard_error  ci_lower  ci_upper
 0  0.040016                 0.009638  0.021124  0.058909
 1 -0.003076                 0.008147 -0.019045  0.012893
 2 -0.097897                 0.037543 -0.171489 -0.024306
 3  0.373015                 0.060843  0.253751  0.492279,
               value  sandwich_standard_errors  ci_lower  ci_upper
 Intercept  0.373011                  0.060851  0.253742  0.492279
 ppltrst    0.040017                  0.009640  0.021122  0.058912
 male      -0.097902                  0.037566 -0.171531 -0.024273
 income    -0.003075                  0.008149 -0.019048  0.012897)